In [181]:
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import ast
import glob
import pingouin as pg

load_dotenv()

data_folder = os.getenv("DATA_FOLDER")

file_prefix = 'true_label_data'
#file_prefix_true = 'true_label_data'

# Use glob to find all CSV files with the given filename start
file_list = glob.glob(f"{data_folder}/{file_prefix}*.csv")
#file_list.extend(glob.glob(f"{data_folder}/{file_prefix_true}*.csv"))

# Read each file into a DataFrame and concatenate them into a single DataFrame
dataframes = [pd.read_csv(file, sep=";",index_col=0) for file in file_list]
df = pd.concat(dataframes, ignore_index=True)

In [182]:
df

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls,newsguard_dict,Labels
0,Day_1_4,R_2y8dRM3TUWjSPIo,Day_1,Misl,0,1,1,20,0,True,...,0.0,0.0,0.0,0.0,92.750000,"factcheck.org', 'thehill.com', 'newsweek.com',...","100,80,100,100,90,92.5,95,87.5,87.5,95",('https://www.factcheck.org/2021/07/debunking-...,"{'factcheck.org': 100, 'thehill.com': 80, 'new...","[{}, {}, {}, {'comparisons': [{'claim_article_..."
1,Day_1_4,R_bqJ3HoBP0lqi7CN,Day_1,True,1,5,3,30,0,True,...,0.0,0.0,0.0,0.0,94.642857,"recorder.maricopa.gov', ''', 'azcentral.com', ...","92.5,100,87.5,95,95,92.5,100",('https://recorder.maricopa.gov/electionresult...,{},"[{}, {}, {'comparisons': [{'claim_article_a': ..."
2,Day_1_4,R_5ovNt0RyjDe4zhD,Day_1,True,1,5,3,37,1,True,...,0.0,0.0,0.0,0.0,95.833333,"azcentral.com', 'abc15.com', 'thehill.com', 'p...","92.5,100,80,100,95,95,100,100,100",('https://www.azcentral.com/story/opinion/op-e...,"{'azcentral.com': 92.5, 'abc15.com': 100, 'the...","[{'comparisons': [{'claim_article_a': ""The lio..."
3,Day_1_4,R_1ILDrpPAAn0CdBx,Day_1,True,1,7,4,45,1,True,...,0.0,1.0,1.0,1.0,84.611111,"thehill.com', 'azcentral.com', 'abc15.com', 'w...","80,92.5,100,42,100,95,100,57,95",('https://thehill.com/homenews/campaign/563658...,"{'thehill.com': 80, 'azcentral.com': 92.5, 'ab...","[{}, {'comparisons': [{'claim_article_a': ""The..."
4,Day_1_4,R_3MycWKTJGEI3fkc,Day_1,True,1,7,4,44,1,True,...,0.0,0.0,0.0,0.0,95.833333,"azcentral.com', 'thehill.com', 'abc15.com', 'b...","92.5,80,100,95,100,95,100,100,100",('https://www.azcentral.com/story/opinion/op-e...,"{'azcentral.com': 92.5, 'thehill.com': 80, 'ab...","[{'comparisons': [{'claim_article_a': ""The lio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Day_12_4,R_vDFQr0QAhdjLCDf,Day_12,True,1,5,3,66,0,True,...,0.0,0.0,0.0,1.0,88.900000,"cnn.com', 'foxnews.com', 'usatoday.com', 'news...","87.5,69.5,100,100,87.5,95,100,69.5,100,80",('https://www.cnn.com/videos/politics/2021/11/...,"{'cnn.com': 87.5, 'foxnews.com': 69.5, 'usatod...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
625,Day_12_4,R_6x2lhuqmicuHOJX,Day_12,True,1,7,4,53,0,True,...,0.0,0.0,0.0,1.0,88.100000,"usatoday.com', 'sfgate.com', 'independent.co.u...","100,100,100,69.5,69.5,80,69.5,100,100,92.5",('https://www.usatoday.com/story/news/politics...,"{'usatoday.com': 100, 'sfgate.com': 100, 'inde...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
626,Day_12_4,R_2WCmKU4ViN13BYG,Day_12,True,1,7,4,39,0,True,...,0.0,0.0,0.0,1.0,89.000000,"news.gallup.com', 'news.gallup.com', 'usatoday...","100,69.5,100,77,87.5,100",('https://news.gallup.com/poll/356591/congress...,"{'news.gallup.com': 69.5, 'usatoday.com': 100,...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
627,Day_12_4,R_3EmSqPx3YNGR6hc,Day_12,Misl,0,3,2,33,1,True,...,0.0,0.0,0.0,1.0,84.050000,"foxnews.com', 'usatoday.com', 'nypost.com', 'c...","69.5,100,69.5,87.5,100,69.5,95,100,80,69.5",('https://www.foxnews.com/opinion/biden-approv...,"{'foxnews.com': 69.5, 'usatoday.com': 100, 'ny...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"


In [183]:
faulty_labels = 0
def safe_literal_eval(x):
    global faulty_labels
    try:
        return ast.literal_eval(x)
    except:
        faulty_labels += 1

df['Labels'] = df['Labels'].apply(safe_literal_eval)
print(faulty_labels)

0


In [184]:
df = df[df['Category'] != 'Coul']

In [185]:
df = df[df['Labels'].notna()]

# Feature Extraction

Features are: stance (headline+position; position); avg_score (Newsguard score); article_position

In [186]:
# taken from the feature_exploration notebook
def retrieve_features(label_dict_list, positions=False, headlines=False):
    #create three distinct lists to control each parameter
    # if label_dict_list is empty return NaN
    if not label_dict_list:
        return np.nan
    article_stances = []
    article_ng_scores = []
    article_serp_positions = []
    
    for article in label_dict_list: #this is the search level; here we can retrieve SERP position and Newsguard score
        if not article: #catch empty entries
            continue
        if len(article['comparisons']) == 0:
               continue
        #retrieve variables
        serp_newsguard = article['serp_newsguard']
        serp_position = article['serp_position']

        #add to lists for decomposed calculation
        article_ng_scores.append(serp_newsguard)
        article_serp_positions.append(serp_position)

        article_stance = 0
        for claim_pair in article['comparisons']: #this is the "article level"; here we can retrieve relative claim position and the bool headline info
            relation = claim_pair['relation']

            if relation == 0: #skip calculation logic for unrelated claims
                continue

            try:
                #retrieve relative claim positions
                fm_claim_position = claim_pair['fm_claim_position']
                serp_claim_position = claim_pair['serp_claim_position']

                #retrieve headline bool
                fm_claim_headline = claim_pair['fm_claim_headline']
                serp_claim_headline = claim_pair['serp_claim_headline']

            except:
                pass

            if not positions and not headlines: #calculate article stance only based on relation
                article_stance += relation
            if positions and not headlines: #calculate article stance based on relation and position
                if relation in [-1, 1]:
                    if serp_claim_position < fm_claim_position: #if serp claim earlier in the article than the respective fm claim
                        article_stance += relation + (serp_claim_position - fm_claim_position) if relation == -1 else relation - (serp_claim_position - fm_claim_position) #e.g. -1 - dif else 1 + dif (reinforcing in each direction)
                    else: # if serp claim later in article than the respective fm claim
                        article_stance += relation - (fm_claim_position - serp_claim_position) if relation == -1 else relation + (fm_claim_position - serp_claim_position) #e.g. - 1 + dif else 1 - dif (weakening in each direction)
            if positions and headlines:
                if relation in [-1, 1]:
                    individual_stance = 0
                    if serp_claim_position < fm_claim_position: #if serp claim earlier in the article than the respective fm claim
                        individual_stance = relation + (serp_claim_position - fm_claim_position) if relation == -1 else relation - (serp_claim_position - fm_claim_position) #e.g. -1 - dif else 1 + dif (reinforcing in each direction)
                        if serp_claim_headline:
                            individual_stance *= 2
                    else: # if serp claim later in article than the respective fm claim
                        individual_stance = relation - (fm_claim_position - serp_claim_position) if relation == -1 else relation + (fm_claim_position - serp_claim_position) #e.g. - 1 + dif else 1 - dif (weakening in each direction)
                        if serp_claim_headline:
                                individual_stance *= 2
                    article_stance += individual_stance
            if headlines and not positions:
                article_stance += relation * serp_claim_headline 
        if headlines and not positions:
            article_stances.append(article_stance/len(article['comparisons']))
        else:
            article_stances.append(article_stance/len(article['comparisons']))

    #if article_stances could not be successfully retrieved, return NaN
    if not article_stances:
        return np.nan
    
    article_ng_scores = np.array(article_ng_scores, dtype=float)
    # article_ng_scores = np.nan_to_num(article_ng_scores, nan=placeholder_ng_score) 

    return {'article_stances': article_stances, 'article_ng_scores': article_ng_scores, 'article_serp_positions': article_serp_positions}

In [187]:
df['Features_Raw']= df['Labels'].apply(retrieve_features)
df['Features_Pos']= df['Labels'].apply(lambda x: retrieve_features(x, positions=True, headlines=False))
df['Features_H']= df['Labels'].apply(lambda x: retrieve_features(x, positions=False, headlines=True))
df['Features_HP']= df['Labels'].apply(lambda x: retrieve_features(x, positions=True, headlines=True))

In [188]:
df['Features_Raw'][1]

{'article_stances': [-0.2, -0.12, 0.04, 0.0, -0.16],
 'article_ng_scores': array([nan, nan, nan, nan, nan]),
 'article_serp_positions': [3, 5, 6, 7, 8]}

In [189]:
#get the mean of all stances
def transform_stances(stances):
    return np.mean(stances)

#assign higher weight to articles higher up in the SERP
def transform_position(positions):
    positions = [1/pos for pos in positions]
    return np.mean(positions)

#transform the newsguard score into the range of 0 to 1
def transform_ng_score(ng_score):
    return ng_score/100

def transform_features(features, ng_score, ng = True, serp_position = True):
    if not isinstance(features, dict):
        return np.nan
    #extract features from feature dict
    stances = features['article_stances']
    positions = features['article_serp_positions']
    #transform stances
    stances = transform_stances(stances)
    #transform positions
    positions = transform_position(positions)
    #transform ng score
    ng_score = transform_ng_score(ng_score)
    features = [stances]
    if serp_position:
        features.append(positions)
    if ng:
        features.append(ng_score)
    if np.isnan(features).any():
        return np.nan
    return features


df['Features_NP_S_pos']= df.apply(lambda x: transform_features(x['Features_Pos'], ng_score=x['avg_score']), axis=1)
df['Features_NP_S_hpos']= df.apply(lambda x: transform_features(x['Features_HP'], ng_score=x['avg_score']), axis=1)
df['Features_N_S_pos']= df.apply(lambda x: transform_features(x['Features_Pos'], ng_score=x['avg_score'], serp_position=False), axis=1)
df['Features_N_S_hpos']= df.apply(lambda x: transform_features(x['Features_HP'], ng_score=x['avg_score'], serp_position=False), axis=1)
df['Features_P_S_pos']= df.apply(lambda x: transform_features(x['Features_Pos'], ng_score=False, ng=False), axis=1)
df['Features_P_S_hpos']= df.apply(lambda x: transform_features(x['Features_HP'], ng_score= False, ng=False), axis=1)

In [190]:
df['Features_NP_S_pos']

0      [-0.04468484493830115, 0.1640873015873016, 0.9...
1      [-0.003744390229209957, 0.19357142857142856, 0...
2      [0.0633824749962681, 0.3983333333333333, 0.958...
3      [0.05157875457875458, 0.27777777777777773, 0.8...
4      [0.05614802585324936, 0.3168367346938776, 0.95...
                             ...                        
624                                                  NaN
625                                                  NaN
626                                                  NaN
627                                                  NaN
628                                                  NaN
Name: Features_NP_S_pos, Length: 629, dtype: object

In [191]:
label_mapping = {
    'Coul' : np.nan,
    'Misl' : 0,
    'True' : 1
}

df['User_Labels'] = df['Category'].map(label_mapping)

In [192]:
df = df.dropna(subset=['User_Labels', 'Features_HP', 'Features_NP_S_pos'])

In [193]:
df.groupby('Category').count()

,Article_day,ResponseId,Day,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,List_Scores,...,Features_Pos,Features_H,Features_HP,Features_NP_S_pos,Features_NP_S_hpos,Features_N_S_pos,Features_N_S_hpos,Features_P_S_pos,Features_P_S_hpos,User_Labels
Category,,,,,,,,,,,,,,,,,,,,,
Misl,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
True,397,397,397,397,397,397,397,397,397,397,...,397,397,397,397,397,397,397,397,397,397


In [203]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, StratifiedKFold


X = np.array(df['Features_NP_S_hpos'].tolist()) #Features_NP_S_pos, Features_NP_S_hpos, Features_N_S_pos, Features_N_S_hpos, Features_P_S_pos, Features_P_S_hpos
y = np.array(df['User_Labels'].tolist())

kf = KFold(n_splits=10, shuffle=True, random_state=1860)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1860)
#loo = LeaveOneOut()

cv_method = skf

model = LogisticRegression(class_weight="balanced")

accuracy = cross_val_score(model, X, y, cv=cv_method, scoring='accuracy') 
precision = cross_val_score(model, X, y, cv=cv_method, scoring='precision') 
recall = cross_val_score(model, X, y, cv=cv_method, scoring='recall') 
f1 = cross_val_score(model, X, y, cv=cv_method, scoring='f1') 

accuracy = cross_val_score(model, X, y, cv=cv_method, scoring='accuracy') 
precision = cross_val_score(model, X, y, cv=cv_method, scoring='precision') 
recall = cross_val_score(model, X, y, cv=cv_method, scoring='recall') 
f1 = cross_val_score(model, X, y, cv=cv_method, scoring='f1') 

print(f"Accuracy: {np.mean(accuracy):.3f}")
print(f"Precision: {np.mean(precision):.3f}")
print(f"Recall: {np.mean(recall):.3f}")
print(f"F1-score: {np.mean(f1):.3f}")

Accuracy: 0.635
Precision: 0.906
Recall: 0.633
F1-score: 0.744


In [195]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

dummy = DummyClassifier(strategy='stratified', random_state=1860)
dummy_accuracy = cross_val_score(dummy, X, y, cv=cv_method, scoring='accuracy')
dummy_precision = cross_val_score(dummy, X, y, cv=cv_method, scoring='precision')
dummy_recall = cross_val_score(dummy, X, y, cv=cv_method, scoring='recall')
dummy_f1 = cross_val_score(dummy, X, y, cv=cv_method, scoring='f1')

baseline_acc = dummy_accuracy.mean()
baseline_precision = dummy_precision.mean()
baseline_recall = dummy_recall.mean()
baseline_f1 = dummy_f1.mean()

print(f"Baseline Accuracy: {baseline_acc:.3f}")
print(f"Baseline Precision: {baseline_precision:.3f}")
print(f"Baseline Recall: {baseline_recall:.3f}")
print(f"Baseline F1: {baseline_f1:.3f}")

Baseline Accuracy: 0.779
Baseline Precision: 0.840
Baseline Recall: 0.912
Baseline F1: 0.874


In [208]:
from scipy import stats

w_stat, p_value = stats.wilcoxon(precision, dummy_precision)
print(f"Wilcoxon test: Statistic: {w_stat:.3f}, P-value: {p_value:.3f}")

Wilcoxon test: Statistic: 1.000, P-value: 0.004
